# Tuning MLPRegressor

In [17]:
import numpy
import csv
import time
import pandas
import signal
import warnings
warnings.filterwarnings("ignore")

In [18]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline

In [19]:
from sklearn.multioutput import ClassifierChain
from sklearn.multioutput import MultiOutputRegressor
from sklearn.multioutput import MultiOutputClassifier

from sklearn.ensemble.weight_boosting import AdaBoostClassifier
from sklearn.ensemble.weight_boosting import AdaBoostRegressor
from sklearn.ensemble.bagging import BaggingClassifier
from sklearn.ensemble.bagging import BaggingRegressor
from sklearn.ensemble.forest import ExtraTreesClassifier
from sklearn.ensemble.forest import ExtraTreesRegressor
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.ensemble.gradient_boosting import GradientBoostingRegressor
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.ensemble.forest import RandomForestRegressor

from sklearn.linear_model.bayes import ARDRegression
from sklearn.linear_model.bayes import BayesianRidge
from sklearn.naive_bayes import BernoulliNB
from sklearn.cross_decomposition.cca_ import CCA
from sklearn.tree.tree import DecisionTreeClassifier
from sklearn.tree.tree import DecisionTreeRegressor
from sklearn.linear_model.coordinate_descent import ElasticNet
from sklearn.tree.tree import ExtraTreeClassifier
from sklearn.tree.tree import ExtraTreeRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process.gpc import GaussianProcessClassifier
from sklearn.gaussian_process.gpr import GaussianProcessRegressor
from sklearn.linear_model.huber import HuberRegressor
from sklearn.neighbors.classification import KNeighborsClassifier
from sklearn.neighbors.regression import KNeighborsRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.semi_supervised.label_propagation import LabelPropagation
from sklearn.semi_supervised.label_propagation import LabelSpreading
from sklearn.linear_model.least_angle import Lars
from sklearn.linear_model.coordinate_descent import Lasso
from sklearn.linear_model.least_angle import LassoLars
from sklearn.linear_model.least_angle import LassoLarsIC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model.base import LinearRegression
from sklearn.svm.classes import LinearSVC
from sklearn.svm.classes import LinearSVR
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.neural_network.multilayer_perceptron import MLPClassifier
from sklearn.neural_network.multilayer_perceptron import MLPRegressor
from sklearn.linear_model.coordinate_descent import MultiTaskElasticNet
from sklearn.linear_model.coordinate_descent import MultiTaskLasso
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.svm.classes import NuSVC
from sklearn.svm.classes import NuSVR
from sklearn.linear_model.omp import OrthogonalMatchingPursuit
from sklearn.cross_decomposition.pls_ import PLSCanonical
from sklearn.cross_decomposition.pls_ import PLSRegression
from sklearn.linear_model.passive_aggressive import PassiveAggressiveClassifier
from sklearn.linear_model.passive_aggressive import PassiveAggressiveRegressor
from sklearn.linear_model.perceptron import Perceptron
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model.ransac import RANSACRegressor
from sklearn.neighbors.classification import RadiusNeighborsClassifier
from sklearn.neighbors.regression import RadiusNeighborsRegressor
from sklearn.linear_model.ridge import Ridge
from sklearn.linear_model.ridge import RidgeClassifier
from sklearn.linear_model.stochastic_gradient import SGDClassifier
from sklearn.linear_model.stochastic_gradient import SGDRegressor
from sklearn.svm.classes import SVC
from sklearn.svm.classes import SVR
from sklearn.linear_model.theil_sen import TheilSenRegressor

In [20]:
mlts = [MultiOutputRegressor, ClassifierChain]
bss = [Ridge, LinearSVR, GaussianProcessRegressor, PassiveAggressiveRegressor, KNeighborsRegressor, MLPRegressor, HuberRegressor, SGDRegressor, LassoLarsIC, ExtraTreeRegressor, ElasticNet, DecisionTreeRegressor, Perceptron, OrthogonalMatchingPursuit, BernoulliNB]

In [21]:
tfidf = pandas.read_csv('07 NM PS TFIDF.csv', delimiter=',', encoding='latin-1').fillna('')
tfidf[:3]

,_____________________,_____________________ follow,able,acceptable,accessory,accessory require,accidentally,accompany,accord,account,...,wrong ct,wrong info,wrong info mri,wrong patient,wrong pt,wrong set,wrong tattoo,xrt,xrt date,xrt pt
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
ohe = pandas.read_csv('07 NM PS OHE.csv', delimiter=',', encoding='latin-1').fillna('')
ohe[:3]

,Treatment delivery,Radiation treatment prescription scheduling,Pre-treatment quality assurance,Post-treatment completion,Interventional procedure for planning and/or delivery,Contouring and planning,Imaging for treatment planning,Patient medical consultation and physician assessment,On-treatment quality assurance
0,1,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0


In [23]:
def lowest_correct(trues, preds):
    num_of_options = len(trues)
    drop_down_options = list(reversed(numpy.argsort(preds)))
    correct_options = [i for i in range(num_of_options) if trues[i]==1]
    return max([drop_down_options.index(correct_option) for correct_option in correct_options]) + 1
def average_lowest_correct(list_of_trues, list_of_preds):
    length = len(list_of_trues)
    return numpy.mean([lowest_correct(list(list_of_trues.iloc[i]), list(list_of_preds[i])) for i in range(length)])

## Default

In [24]:
def deadline(timeout, *args):
    def decorate(f):
        def handler(signum, frame):
            raise Exception

        def new_f(*args):
            signal.signal(signal.SIGALRM, handler)
            signal.alarm(timeout)
            return f(*args)
            signal.alarm(0)

        new_f.__name__ = f.__name__
        return new_f
    return decorate

In [25]:
@deadline(500)
def cv_ensemble(model, xs, ys):
    temp = cross_validate(model, xs, ys, return_train_score=True, scoring=make_scorer(average_lowest_correct), n_jobs=1, cv=2)
    return numpy.mean(temp['test_score']).round(2)

In [26]:
cv_ensemble(MultiOutputRegressor(MLPRegressor()), tfidf, ohe)

1.97

## Grid Search

In [27]:
def prn_rsl(gsc):
    prms = gsc.cv_results_['params']
    tsts = (gsc.cv_results_['split0_test_score']+gsc.cv_results_['split1_test_score'])/2
    for i in numpy.argsort(tsts):
        print(prms[i], tsts[i])

In [28]:
ppl = Pipeline([('mb', MultiOutputRegressor(MLPRegressor()))])

In [ ]:
grd = {
    'mb__estimator__hidden_layer_sizes': [(i) for i in range(100, 500, 100)],
}
gsc = GridSearchCV(estimator=ppl, param_grid=grd, scoring=make_scorer(average_lowest_correct), n_jobs=1, cv=2, verbose=0)
gsc.fit(tfidf, ohe)
prn_rsl(gsc)

In [ ]:
grd = {
    'mb__estimator__hidden_layer_sizes': [(i, i) for i in range(500, 1000, 100)],
} 
gsc = GridSearchCV(estimator=ppl, param_grid=grd, scoring=make_scorer(average_lowest_correct), n_jobs=1, cv=2, verbose=0)
gsc.fit(tfidf, ohe)
prn_rsl(gsc)